# Colab Root User

## 注意

このノートブックはColab課金ユーザのみ実行可能です。

無課金ユーザはアカウント停止の原因になります。

実行前に[利用規約](https://research.google.com/colaboratory/faq.html#disallowed-activities)を確認してください。

## 事前準備

ローカルPCでSSHキーを作成

```sh
# ターミナルで実行
ssh-keygen -t ed25519 -C "your_email@example.com"
# または RSA の場合
ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

# Enter キーを3回押す（デフォルト設定の場合）
# パスフレーズを設定することも可能
```

公開鍵の確認

```sh
cat ~/.ssh/id_ed25519.pub
# または
cat ~/.ssh/id_rsa.pub
出力例:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAI... your_email@example.com
```

## SSHサーバの起動

In [ ]:
# 公開鍵を取得
from google.colab import userdata
PUBLIC_KEY = userdata.get('PUBLIC_KEY')
assert PUBLIC_KEY

# SSH ディレクトリの作成
ssh_dir = "/root/.ssh"
!mkdir -p {ssh_dir}
!chmod 700 {ssh_dir}

# authorized_keys に書き込み
authorized_keys_path = f"{ssh_dir}/authorized_keys"
with open(authorized_keys_path, 'w') as f:
    f.write(PUBLIC_KEY.strip())

# パーミッション設定
!chmod 600 {authorized_keys_path}

# SSHサーバーのインストールと設定
print("📦 SSH サーバーをインストールしています...")
!apt-get update -qq
!apt-get install -y openssh-server > /dev/null 2>&1

# SSH設定ファイルを作成（rootログイン許可）
sshd_config = """Port 22
PermitRootLogin prohibit-password
PubkeyAuthentication yes
PasswordAuthentication no
ChallengeResponseAuthentication no
UsePAM yes
X11Forwarding yes
PrintMotd no
AcceptEnv LANG LC_*
Subsystem sftp /usr/lib/openssh/sftp-server
"""

with open('/etc/ssh/sshd_config', 'w') as f:
    f.write(sshd_config)

# SSHサービスを起動
!service ssh start
!service ssh status | grep Active

print("\n✅ SSH サーバーが起動しました")
print("   - ルートログイン: 公開鍵認証のみ許可")
print("   - パスワード認証: 無効")
print("   - 公開鍵認証: 有効\n")

# ユーザー情報を表示
print(f"📋 ユーザー情報:")
!id root

## 環境構築

In [ ]:
# Rust インストール
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

# Claude Codeをインストール
!curl -fsSL https://claude.ai/install.sh | bash && \
    echo 'export PATH="$HOME/.claude/bin:$PATH"' >> /root/.bashrc && \
    echo 'alias claude="claude --dangerously-skip-permissions"' >> /root/.bashrc

# Codex CLIをインストール
!npm i -g @openai/codex

# Gemini CLIをインストール
!npm install -g @google/gemini-cli

# GitHub Copilot CLIをインストール
!npm install -g @github/copilot

# .tmux.confを設定
!curl -fsSL https://raw.githubusercontent.com/hayatoshibahara/tmux/refs/heads/main/.tmux.conf -o /root/.tmux.conf

# .bashrcを設定
!echo 'cd /content' >> /root/.bashrc

## Cloudflaredの起動

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /tmp/cloudflared
!chmod +x /tmp/cloudflared
!/tmp/cloudflared tunnel --url ssh://localhost:22

トンネルURLが「https://comparing-trim-background-improve.trycloudflare.com」の場合、以下をローカルPCで設定する。

~/.ssh/config に追加:

```sh
Host colab
HostName comparing-trim-background-improve.trycloudflare.com
User root
ProxyCommand cloudflared access ssh --hostname %h
IdentityFile ~/.ssh/id_ed25519
IdentitiesOnly yes
StrictHostKeyChecking no
UserKnownHostsFile /dev/null
ServerAliveInterval 60
ServerAliveCountMax 3
```

これにより `ssh colab` やVSCodeでSSH接続できる。